In [5]:
# !pip install litellm==1.26.0
import os
import litellm
import sys
from pathlib import Path
import numpy as np
import pandas as pd
from collections import OrderedDict

In [6]:
# [OPTIONAL] set env var
api_key = os.environ['OPENAI_API_KEY'] # string, you need your own key and to put at least $5 in the account
#s.environ["COHERE_API_KEY"] = api_keys.cohere_trial # string, you can get for free by logging into cohere and going to sandbox

gpt4_turbo = "gpt-4-1106-preview"
cohere = 'command-nightly'
litellm.drop_params=True # will ignore paramaters you set if they don't belong in a model

In [7]:
data_root = Path("/Volumes/Samsung_T5/MIT/abcd/release_05/abcd-data-release-5.1")
vocab_path  = Path("/Volumes/Samsung_T5/MIT/abcd/adhd_vocab1.npy")

section = "core/mental-health"
table = 'mh_p_ksads_adhd.csv'
mapping = 'ksads_adhd.csv'

data_section = Path( data_root, section)
data_path = Path(data_section, table) # path to results table  
var_mapping_path =  Path(data_root, "var_mapping", mapping) # path to mapping of variable names to questions 

data = pd.read_csv(data_path)
var_description = pd.read_csv(var_mapping_path)
symptoms = (np.load(vocab_path, allow_pickle=True))

var_names = np.array(var_description["var_name"])
questions = np.array(var_description["var_label"])
mapping = OrderedDict(zip(var_names, questions))


/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_12159/1068665958.py:12: DtypeWarning: Columns (23,76,83,84) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(data_path)


In [8]:
symptoms

array(['self-inflicted stress exposure', 'hostile agression',
       'unstable sleep habits', 'anhedhonia', 'frustrative non-reward',
       'Reward anticipation impairment', 'non-perseverance',
       'forgetfulness', 'time perception',
       'adaptation to new enviroments ', 'cognitive flexibility',
       'attentional lapses', 'motor overactivity',
       'fine motor coordination impairement', 'risk taking ',
       'impatience', 'substance abuse', 'distractability',
       'behavioral inhibition', 'non-plannfulness', 'implusivity'],
      dtype=object)

In [9]:
# from openai import OpenAI
# client = OpenAI()
# completion = client.chat.completions.create(
#   model="gpt-3.5-turbo",
#   messages=[
#     {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
#     {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
#   ]
# )

In [10]:
from litellm import completion


def api_request(
    prompt,
    model="commmand-nightly",
    api_key=None,
    temperature=0.1,
    top_p=1,
    timeout=45,
    num_retries=2,
    max_tokens=None,
    seed=None,
    response_format=None,
):
    # Open AI status: https://status.openai.com/

    messages = [{"content": prompt, "role": "user"}]
    responses = completion(
        model=model,
        messages=messages,
        api_key=api_key,
        temperature=temperature,
        top_p=top_p,
        timeout=timeout,
        num_retries=num_retries,
        max_tokens=max_tokens,
        seed=seed,
        # response_format = response_format
    )
    response = responses.get("choices")[0].get("message").get("content")  # access response for first message
    return response



In [11]:
questions

array(['In the past two weeks, how often has your child put off or avoided doing things that were hard for him or her because they required a lot of attention?',
       'Do the problems we have just been talking about cause difficulties for your child in any of the following areas:With family',
       'Do the problems we have just been talking about cause difficulties for your child in any of the following areas:With friends',
       "In the past two weeks, how often has your child talked when he or she wasn't supposed to, or had people complain that he or she talked too much?",
       'In the past two weeks, how often has your child interrupted someone when they were talking?',
       'In the past two weeks, how often has your child had trouble waiting for his or her turn or waiting for other things?',
       "In the past two weeks, how often has your child blurt out answers in school without being called on, or blurt out answers when someone wasn't talking to him or her?",
       'In

In [12]:
prompt = """
Paraphrase the following question 10 times. Make sure to use words that are as different as possible from other paraphrasings as possible: {sentence}.
 Don't give any additional explanation.
"""

In [16]:

prompt = prompt.format(sentence = questions[0])
response = api_request(prompt, model=gpt4_turbo, temperature=0)
print(response)


1. Over the last fortnight, how frequently has your youngster procrastinated or shunned tasks that were challenging due to their demand for intense focus?
2. In the recent 14 days, to what extent has your offspring delayed or bypassed activities that were difficult and necessitated considerable concentration?
3. How many times in the preceding two weeks has your kid evaded or postponed engaging in demanding activities that called for significant mental effort?
4. During the past couple of weeks, what has been the frequency of your child's tendency to defer or steer clear of strenuous tasks that required heightened attentiveness?
5. Can you quantify how often your child has shirked or put on hold difficult tasks needing substantial focus in the last fourteen days?
6. In the previous fortnight, has your child frequently avoided or delayed tasks that were tough and needed a lot of focus?
7. How regularly in the past two weeks has your child sidestepped or procrastinated on assignments tha

In [14]:
from sentence_transformers import SentenceTransformer
embeddings_name = "all-MiniLM-L6-v2"
sentence_embedding_model = SentenceTransformer(embeddings_name)       # load embedding
sentence_embedding_model._first_module().max_seq_length = 500 # if you want to increase, but makes it slower
print(sentence_embedding_model.max_seq_length) #default = 256

500


In [15]:
embeddings_qst_test = sentence_embedding_model.encode(test_qst, convert_to_tensor=True,show_progress_bar=True)
embeddings_vocab = sentence_embedding_model.encode(vocab, convert_to_tensor=True, show_progress_bar=True)

NameError: name 'test_qst' is not defined

In [ ]:
# prompt = """
# Classify the document into one of the following symptoms:
# Symptoms to choose from: {symptoms}

# Output only the most likely symptom. As an explanation, give me phrase that help you classify this document. Also give me the confidence level for the most likley symptom, as a number between 0 and 1. 
# I will not give a confidence level for my examples, but you should provide it. Don't give any additional explanation.

# For example, 
# Document: {example_1}
# Symptom: {symptom_1}
# Explanation: {explanation_1}
# Confidence level: 


# Document: {example_2}
# Symptom: {symptom_2}
# Explanation: {explanation_1}
# Confidence level: 

# Document: {document}
# Symptom:
# Explanantion: 
# Confidence level: 
# """

In [ ]:
#symptoms = ['sustained attention', 'hyperactivity', 'disruptive behavior', 'impulse control', 'reward anticipation', 'agression']
symptoms_str = '; '.join(symptoms)
symptoms_str

'self-inflicted stress exposure; hostile agression; unstable sleep habits; anhedhonia; frustrative non-reward; Reward anticipation impairment; non-perseverance; forgetfulness; time perception; adaptation to new enviroments ; cognitive flexibility; attentional lapses; motor overactivity; fine motor coordination impairement; risk taking ; impatience; substance abuse; distractability; behavioral inhibition; non-plannfulness; implusivity'

In [ ]:
examples = ['In the past two weeks, how often has your child interrupted someone when they were talking?', 
            'In the past two weeks, how often has your child been on the go, moving from one place to another?',
            'You said that in the past your child had a lot of trouble keeping focused and paying attention; was often easily distracted; often had trouble staying seated; was impulsive and often acted before he or she thought. When was that?']

symptoms_examples = ['behavioral inhibition', 
                     'motor overactivity']
#documents = ["In the past two weeks, I interrupted someone while talking"]
explanations = ['interrupted someone when they were talking',
                 "on the go, moving from one place to another"
                ]
responses = {}
for i, document in enumerate(questions):
    prompt_i = prompt.format(symptoms = symptoms_str, example_1 = examples[0], example_2 = examples[1], symptom_1 = symptoms_examples[0], symptom_2 = symptoms_examples[1], explanation_1 = explanations[0], explanation_2 = explanations[1], document = document)
    response = api_request(prompt_i, model=gpt4_turbo)
    print(document)
    print(response)
    print('\n')
    responses[i] = response

In the past two weeks, how often has your child put off or avoided doing things that were hard for him or her because they required a lot of attention?
Symptom: non-plannfulness
Explanation: put off or avoided doing things that were hard
Confidence level: 0.85


Do the problems we have just been talking about cause difficulties for your child in any of the following areas:With family
Symptom: behavioral inhibition
Explanation: cause difficulties for your child in any of the following areas: With family
Confidence level: 0.85


Do the problems we have just been talking about cause difficulties for your child in any of the following areas:With friends
Symptom: social impairment
Explanation: cause difficulties for your child with friends
Confidence level: 0.85


In the past two weeks, how often has your child talked when he or she wasn't supposed to, or had people complain that he or she talked too much?
Symptom: impulsivity
Explanation: talked when he or she wasn't supposed to, or had pe

In [ ]:
response.split('\n')

NameError: name 'response' is not defined

In [ ]:
def generate_prompt(
    construct,
    prompt_name=None,
    prompt="default",
    domain=None,
    definition=None,
    examples=None,
    output_format="default",
    remove_parentheses_definition=True,
):
    if output_format == "default":
        output_format = (
            "Each token should be separated by a semicolon. Do not return duplicate tokens. Do not provide any"
            " explanation or additional text beyond the tokens."
        )
    # removed: Order them by how similar they are to {construct}.
    elif output_format == "json":
        output_format = (
            "Provide tokens in JSON output. Do not return duplicate tokens. Do not provide any explanation or"
            " additional text beyond the tokens."
        )

    # Prompt
    if not isinstance(prompt_name, str):
        # if prompt_name == None:
        prompt_name = construct.replace("_", " ").lower()

    if prompt == "default":
        prompt = "Provide many single words and some short phrases related to"
        if domain:
            domain = f"(in the {domain} domain). "
            prompt = f"""{prompt} {prompt_name} {domain}{output_format}"""
        else:
            prompt = f"""{prompt} {prompt_name}. {output_format}"""
        if definition:
            if remove_parentheses_definition:
                definition = re.sub(r"\(.*?\)", "", definition)  # remove parentheses which is usually citation.
            prompt += f"\nHere is a definition of {prompt_name}: {definition.lower().strip()}"

        if isinstance(examples, list):
            examples = "; ".join(examples)
        if isinstance(examples, str):
            # examples = '; '.join(examples)
            prompt += f"\nHere are some examples (include these in the list): {examples}."

    return prompt

